In [ ]:
# GENERIC FIRST CELL FOR DEVELOPING A NEW METHODOLOGY

# Define an expressive name for the methodology, that will be used to create the output directory
# The complete path to the output directory will be provided in OUT_DIR
METHODOLOGY_NAME='pos_log/'

import os
OUT_DIR = os.path.relpath(os.path.join(os.getcwd(), 'out', METHODOLOGY_NAME))
if not os.path.exists(OUT_DIR):
        os.makedirs(OUT_DIR)
print(f"Current output directory: '{OUT_DIR}'")

%load_ext autoreload
%autoreload 2

In [ ]:
# POS_LOG_MODEL='pos_log_upos_crf_10k_model_pre_manual'

# # Precons
# INPUT_FILE=os.path.join(OUT_DIR, '..','numb_var','examples_10000_each_seed-42_numb_var.csv')

# # if file does not exist, throw an error
# if not os.path.exists(INPUT_FILE):
#     precon='1_templates_collect_numb_var.ipynb'
#     raise FileNotFoundError(f"File '{INPUT_FILE}' not found. Run '{precon}' first.")

# OUTPUT_FILE=os.path.join(OUT_DIR, 'examples_tagged_upos.csv')
# print(f'Output file: {OUTPUT_FILE}')

# RANDOM_SAMPLE_SIZE=100
# RANDOM_SAMPLE_SEED=42
# OUTPUT_FILE_RANDOM_SAMPLE=os.path.join(OUT_DIR, f'examples_tagged_{RANDOM_SAMPLE_SIZE}_random_sample_seed-{RANDOM_SAMPLE_SEED}_upos.csv')
# print(f'Output file: {OUTPUT_FILE_RANDOM_SAMPLE}')

In [ ]:
# import pandas as pd
# # columns: Dataset,Line,Example,Template,ClusterId
# example_df=pd.read_csv(INPUT_FILE)
# example_df

In [ ]:
# tagged_sents=[]
# for word_i in full_tagged_indices:
#     tokens=examples_splitted[word_i]
#     tags=majorities[word_i]
#     tagged_sents.append(list(zip(tokens,tags)))

### Train First CRF

In [ ]:
from nlp.pos import PosLogCRF
crf=PosLogCRF(model_path=POS_LOG_MODEL)
crf.train_from_tagged_sents(tagged_sents)


# Now tag with all taggers not fully corrected lines and evaluate while skipping None Values in Tags
# This should give an Comparison of PosLog and Taggers

In [ ]:
# these are all lines and their counted majority containing Nones
majorities

In [ ]:
# line indices that was not trained on
len(ragged_tagged_indices)

In [ ]:
ragged_lines = [examples_splitted[i] for i in ragged_tagged_indices]
ragged_majorities = [majorities[i] for i in ragged_tagged_indices]
X=ragged_lines
y=ragged_majorities
#ragged_zipped = [list(zip(line, majority)) for line, majority in zip(ragged_lines, ragged_majorities)]
#ragged_zipped

In [ ]:
y_pred=[crf.predict(x) for x in X]

In [ ]:
def remove_none_values_from_ys(y, y_pred):
    yas=[]
    yps=[]
    for i in range(len(y)):
        ya=[]
        yp=[]
        for j in range(len(y[i])):
            #if y[i][j] is not None:# and y[i][j]!='PROMETEUS':
            if y[i][j] is not None and not y[i][j].startswith('PROMETEUS'):
                ya.append(y[i][j])
                yp.append(y_pred[i][j])
        yas.append(ya)
        yps.append(yp)
    return yas, yps

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

def print_accuracy(y, y_pred, report=False)->float:
    yas, yps = remove_none_values_from_ys(y, y_pred)
    y_t=[tag for tags in yas for tag in tags]
    y_p=[tag for tags in yps for tag in tags]

    acc=accuracy_score(y_t, y_p)
    print(acc)
    if report:
        print(classification_report(y_t, y_p))
    return acc



In [ ]:
import time
def tag_and_meas_time(tagger:AbstractPosTagger, X:list[list[str]])->tuple[list[list[str]], float]:
    time_start = time.time()
    y_pred=[tagger.pos_tag(x) for x in X]
    elapsed_time=time.time()-time_start
    return y_pred, elapsed_time

In [ ]:
def tag_and_evaluate(tagger:AbstractPosTagger, X:list[list[str]], y:list[list[str]], last_pred:tuple[list[list[str]],float]|None=None, report=False)->list[list[str]]:
    print(tagger.__class__.__name__)

    if last_pred is None:
        y_pred,elapsed_time=tag_and_meas_time(tagger, X)
    else:
        y_pred,elapsed_time=last_pred

    acc=print_accuracy(y, y_pred, report)
    lines=len(X)
    tokens=sum([len(x) for x in X])
    print(f"Lines: {lines}")
    print(f"Tokens: {tokens}")
    print(f"Elapsed time: {elapsed_time}")
    print(f"Time per 1000 lines: {elapsed_time/lines*1000}")
    print(f"Time per 1000.000 tokens: {elapsed_time/tokens*1000000}\n")
    return (y_pred, elapsed_time, acc)


In [ ]:
x=tag_and_evaluate(crf, X, y, report=True)


In [ ]:
poslog=PosLogCRF(model_path=POS_LOG_MODEL)
x=tag_and_evaluate(poslog, X, y, report=True)
#x=tag_and_evaluate(crf, X, y, report=True)

In [ ]:
y_pred_nltk=tag_and_evaluate(nltk_tagger, X, y)
y_pred_stanza=tag_and_evaluate(stanza_tagger, X, y)
y_pred_spacy=tag_and_evaluate(spacy_tagger, X, y)
y_pred_hanta=tag_and_evaluate(hanta_tagger, X, y)
y_pred_treetagger=tag_and_evaluate(treetagger_tagger, X, y)

In [ ]:
# poslog=PosLogCRF(model_path=POS_LOG_MODEL)
# x=tag_and_evaluate(poslog, X, y, report=True)
# #x=tag_and_evaluate(crf, X, y, report=True)

In [ ]:
t = example_df['Tokens'].tolist()[9]
print(t)


for name, tagger_func in tagger.items():
    if name=='pos_log':
        tagger_func=poslog.pos_tag
    #pos_tags = poslog.pos_tag(tokenizer.tokenize(s))
    #t=tokenizer.tokenize(s)
    pos_tags = tagger_func(t)
    #pos_tags = [tag for token, tag in pos_tags]
    print(name, list(zip(t, pos_tags)))


In [ ]:
print_accuracy(y, y_pred)

In [ ]:
print(ragged_tagged_indices)
print(full_tagged_indices)

In [ ]:
only_ds_indices={i:None for i in example_df[example_df['Dataset']=='HDFS'].index.tolist()}
len(only_ds_indices)

In [ ]:
from nlp.pos import PosLogCRF

def exclude_ds_and_train(dataset:str):
    datasets=example_df['Dataset'].unique().tolist()

    all_but_ds_indices={i:None for i in example_df[example_df['Dataset']!=dataset].index.tolist()}
    only_ds_indices={i:None for i in example_df[example_df['Dataset']==dataset].index.tolist()}

    train_datasets=[d for d in datasets if d!=dataset]
    print(f"Training on {train_datasets} ({len(all_but_ds_indices)} lines) and excluding {dataset} ({len(only_ds_indices)} lines)")

    tagged_sents_ex=[]
    for i in full_tagged_indices:
        if i in only_ds_indices:
            continue
        tokens=examples_splitted[i]
        tags=majorities[i]
        tagged_sents_ex.append(list(zip(tokens,tags)))
    pos_log_ex=PosLogCRF(model_path='ex_'+dataset+'_'+POS_LOG_MODEL)
    pos_log_ex.train_from_tagged_sents(tagged_sents_ex)

    y_ex=[]
    X_ex=[]
    for i in ragged_tagged_indices:
        if i in all_but_ds_indices:
            continue
        X_ex.append(examples_splitted[i])
        y_ex.append(majorities[i])

    print(f"Trained on {len(tagged_sents_ex)} lines, evaluating on {len(X_ex)} lines")
    x_ex=tag_and_evaluate(pos_log_ex, X_ex, y_ex)#, report=True)

    # y_ex=[]
    # X_ex=[]
    # for i in ragged_tagged_indices:
    #     X_ex.append(examples_splitted[i])
    #     y_ex.append(majorities[i])
    # print(f"All datasets lines: {len(X_ex)}")
    # x_all=tag_and_evaluate(pos_log_ex, X_ex, y_ex)#, report=True)
    return x_ex

x=exclude_ds_and_train('HDFS')


In [ ]:
from nlp.pos import PosLogCRF

datasets=example_df['Dataset'].unique().tolist()

exclude_comparisons=pd.DataFrame(datasets, columns=['Dataset'])

accs=[]
for dataset in datasets:
    print(f"\nDataset: {dataset}")
    x=exclude_ds_and_train(dataset)
    accs.append(x[2])
exclude_comparisons['poslog'] = accs
    

In [ ]:
exclude_comparisons

In [ ]:

tagger_objects=[nltk_tagger, stanza_tagger, spacy_tagger, hanta_tagger, treetagger_tagger]
for tagger_x in tagger_objects:
    accs=[]
    for dataset in datasets:
        #tagger_x=tagger

        print(f"\nDataset: {dataset}")
        datasets=example_df['Dataset'].unique().tolist()

        all_but_ds_indices={i:None for i in example_df[example_df['Dataset']!=dataset].index.tolist()}
        only_ds_indices={i:None for i in example_df[example_df['Dataset']==dataset].index.tolist()}

        train_datasets=[d for d in datasets if d!=dataset]
        print(f"Training on {train_datasets} ({len(all_but_ds_indices)} lines) and excluding {dataset} ({len(only_ds_indices)} lines)")

        tagged_sents_ex=[]
        for word_i in full_tagged_indices:
            if word_i in only_ds_indices:
                continue
            tokens=examples_splitted[word_i]
            tags=majorities[word_i]
            tagged_sents_ex.append(list(zip(tokens,tags)))

        y_ex=[]
        X_ex=[]
        for word_i in ragged_tagged_indices:
            if word_i in all_but_ds_indices:
                continue
            X_ex.append(examples_splitted[word_i])
            y_ex.append(majorities[word_i])

        print(f"Trained on {len(tagged_sents_ex)} lines, evaluating on {len(X_ex)} lines")
        x_ex=tag_and_evaluate(tagger_x, X_ex, y_ex)#, report=True)

        accs.append(x_ex[2])
    exclude_comparisons[tagger_x.__class__.__name__] = accs

    # y_ex=[]
    # X_ex=[]
    # for i in ragged_tagged_indices:
    #     X_ex.append(examples_splitted[i])
    #     y_ex.append(majorities[i])
    # print(f"All datasets lines: {len(X_ex)}")
    # x_all=tag_and_evaluate(pos_log_ex, X_ex, y_ex)#, report=True)
    #return x_ex


In [ ]:

accs=[]
for dataset in datasets:
    tagger_x=poslog

    print(f"\nDataset: {dataset}")
    datasets=example_df['Dataset'].unique().tolist()

    all_but_ds_indices={i:None for i in example_df[example_df['Dataset']!=dataset].index.tolist()}
    only_ds_indices={i:None for i in example_df[example_df['Dataset']==dataset].index.tolist()}

    train_datasets=[d for d in datasets if d!=dataset]
    print(f"Training on {train_datasets} ({len(all_but_ds_indices)} lines) and excluding {dataset} ({len(only_ds_indices)} lines)")

    tagged_sents_ex=[]
    for word_i in full_tagged_indices:
        if word_i in only_ds_indices:
            continue
        tokens=examples_splitted[word_i]
        tags=majorities[word_i]
        tagged_sents_ex.append(list(zip(tokens,tags)))

    y_ex=[]
    X_ex=[]
    for word_i in ragged_tagged_indices:
        if word_i in all_but_ds_indices:
            continue
        X_ex.append(examples_splitted[word_i])
        y_ex.append(majorities[word_i])

    print(f"Trained on {len(tagged_sents_ex)} lines, evaluating on {len(X_ex)} lines")
    x_ex=tag_and_evaluate(tagger_x, X_ex, y_ex)#, report=True)

    accs.append(x_ex[2])

exclude_comparisons[crf.__class__.__name__] = accs



In [ ]:
#exclude_comparisons = exclude_comparisons.drop(index='average', errors='ignore')
exclude_comparisons


In [ ]:

exclude_comparisons.loc['average'] = exclude_comparisons.drop(columns=['Dataset']).mean(axis=0)
exclude_comparisons

In [ ]:
# TODO: Add PosLog trained on all datasets (not excluding)

In [ ]:
# print all parity tokens
c=0
l=[]
for tokens,tc in zip(examples_splitted,tag_comparisons):
    for word_i,maj in enumerate(tc.majority):
        if tc.confidence[word_i]<=0.6:
            word_kind=known_words_det.kind_of_known_word(tokens[word_i])
            if word_kind!=WordKind.UNKNOWN:
                continue
            c+=1
            l.append(tokens[word_i])
            print(f"Token: {tokens[word_i]} Majority: {tc.majority[word_i]} Minority: {tc.minority[word_i]}")
print(f"Total: {c}")

In [ ]:
s='i have to walk'
t=tokenizer.tokenize(s)
for name, func in tagger.items():
    print(f"{name}: {func(t)}")


In [ ]:
print(len(set(l)))
sorted(list(set(l)))

In [ ]:
print(matcher.token_class('lol'))
for t in sorted(list(set(l))):
    print(t,matcher.token_class(t))


In [ ]:
s='his hostname was Noname'
for name, func in tagger.items():
    print(func(tokenizer.tokenize(s)))

In [ ]:
# if Prometeus said it was PUNCTUATION, then correct all tags to PUNCT with confidence 1.0
import string
import re
def override_punctuation_tags(tokens:list[str], tag_comparison:TagComparison)->TagComparison:
    for i,token in enumerate(tokens):
        #if token in string.punctuation:
        # since string.punctuation is defined as raw string, we can use it directly without matching e.g. '.*'
        if re.fullmatch(r'['+string.punctuation+']+', token):
            tag_comparison.majority[i]='PUNCT'
            tag_comparison.confidence[i]=1.0
            tag_comparison.minority[i]={}
    return tag_comparison

tag_comparisons_masked=[override_punctuation_tags(tokens, tc) for tokens,tc in zip(masked_examples, tag_comparisons_masked)]
print('Masked')
print_stats_on_tag_comparison(tag_comparisons_masked)
print()
print()
print('Unmasked')
tag_comparisons_unmasked=[override_punctuation_tags(tokens, tc) for tokens,tc in zip(masked_examples, tag_comparisons_unmasked)]
print_stats_on_tag_comparison(tag_comparisons_unmasked)


In [ ]:
# filter example_df for 

In [ ]:
# save to df
example_df['TagComparison']=tag_comparisons_unmasked
example_df['TagComparisonMasked']=tag_comparisons_masked
#write file
example_df.to_csv(OUTPUT_FILE, index=False)
example_df.head()


In [ ]:
s='hostname'
s='?'
mask=masker.get_token_class(s)
print(f'Masked: {mask}')
print(f'Is masked? {masker.is_masked_token(mask)}')

In [ ]:
######
# UPOS
######
# for nltk we need our own mapping or use the "repaired" one
# stanza does full upos :)
# spacy looks good, too

# hanta we need to repair the mapping
# treetagger we need our own mapping, but we can use "repaired" nltk's mapping again

######
# PTB
######
# nltk does ptb
# stanza does ptb (most likely)
# spacy does ptb (most likely)
# hante we need to build brown2ptb mapping
# tree tagger uses ptb, but we may have to fix some tags


s="It's not easy to find words, because it needs 10 $ at the door which had died. And Reagen has died, too. I give up trying to get a particle. But hey, there it is ;) Wow!"
#s='session opened for user cyrus by (uid=0)'
print(nltk_tagger.pos_tag_ptb(tokenizer.tokenize(s)))
print(nltk_tagger.pos_tag_upos(tokenizer.tokenize(s)))
print(stanza_tagger.pos_tag_upos(tokenizer.tokenize(s)))
print(spacy_tagger.pos_tag_upos(tokenizer.tokenize(s)))
print(hanta_tagger.pos_tag_upos(tokenizer.tokenize(s)))
print(treetagger_tagger.pos_tag_upos(tokenizer.tokenize(s)))

print(tokenizer.tokenize(s))
ttts=sorted(set(spacy_tagger.pos_tag_upos(tokenizer.tokenize(s))))
print(len(ttts))
print(ttts)

tttsn=sorted(set(hanta_tagger.pos_tag_upos(tokenizer.tokenize(s))))
print(len(tttsn))
print(tttsn)

In [ ]:
import string

s="It's `really' \"not\" easy -- to ``find'' 100 words rather short than that, because it needs 10 $ at the door which had died. And Reagen has died, too. I give up trying to get a particle. But hey, there it is ;) Wow!"
#s=' . '.join(list(string.punctuation))
print(s)
ts=tokenizer.tokenize(s)
ts2=zip(ts, nltk_tagger.pos_tag_upos(ts), [t[2] for t in hanta_tagger.nlp.tag_sent(ts, taglevel=1)])
ts2=zip(ts, treetagger_tagger.pos_tag_ptb(ts), [t[2] for t in hanta_tagger.nlp.tag_sent(ts, taglevel=1)])
ts2=zip(ts, spacy_tagger.pos_tag_upos(ts), stanza_tagger.pos_tag_upos(ts), treetagger_tagger.pos_tag_upos(ts), #treetagger_tagger.pos_tag_ptb(ts), 
        hanta_tagger.pos_tag_upos(ts), nltk_tagger.pos_tag_upos(ts))
#from treetaggerwrapper import Tag, NotTag, make_tags
#tsss=treetagger_tagger.nlp.tag_text(ts, nosgmlsplit=True, notagurl=True, notagemail=True, notagip=True, notagdns=True, tagonly=True)
#ts2=zip(ts, make_tags(tsss), [t[2] for t in hanta_tagger.nlp.tag_sent(ts, taglevel=1)])
for t in ts2:
    if t[1]!=t[2] or t[1]!=t[3] or t[2]!=t[3]:
        print(t)

#treetagger_tagger.nlp.tagbin


In [ ]:
from nlp.pos.tag_comparison import TagComparison, make_tag_comparison, print_stats_on_tag_comparison
s="I (give) up {trying} to [get] get a particle. But hey, there it is ;) Wow!"
from string import punctuation
s=' . '.join(list(punctuation))
print(s)
ts=tokenizer.tokenize(s)
taggers_res={}
print(f"Tokens: {ts}")


TAGSET='upos'
#TAGSET='ptb'
for name, func in tagger.items():
    taggers_res[name]=func(ts,TAGSET)
    #print(f"{name}: {taggers_res[name]}")
    print(f"{name}: {taggers_res[name]}")
tc=make_tag_comparison(taggers_res, TAGSET)
print(tc)
for word_i in range(len(ts)):
    tcc=tc.confidence[word_i]
    if tcc<1.0:
        print(f"{ts[word_i]}: {tc.majority[word_i]}")
        print(f"{ts[word_i]}: {tc.minority[word_i]}")

In [ ]:
from collections import Counter
c=Counter({'a':1, 'b':2, 'c':3})
sorted(c.values(), reverse=True)

In [ ]:
a="{\"uid\":\"MsMC0pc\",\"token\":null,\"ma\":null,\"question\":[{\"questionId\":62377,\"answerId\":244003,\"answerText\":\"Pinte\n\",\"answer\":1}],\"iframe\":true,\"ref\":\"www.dewezet.de\",\"ts\":$(date +%s)}"
a="{\"uid\":false,\"token\":null,\"ma\":null,\"question\":[{\"questionId\":62377,\"answerId\":244003,\"answerText\":\"Pinte\n\",\"answer\":1}],\"iframe\":true,\"ref\":\"www.dewezet.de\",\"ts\":$(date +%s)}"
a="https://aktion.dewezet.de/umfrage/g4kyv/auswertung/?q=1&uid=Noh9J2K&iframe&ref=www.dewezet.de&adUnit1=&adUnit2=&pageIVWCat=0"
a

In [ ]:
s='BLOCK* NameSystem.addStoredBlock: blockMap updated: 10.250.10.6:50010 is added to blk_-1608999687919862906 size 91178'
s='PacketResponder 0 for block blk_-3544583377289625738 terminating'
s='Received block blk_-9073992586687739851 of size 11977 from /10.250.7.244'
print(s)
print(tokenizer.tokenize(s))
for t_n, t in tagger.items():
    print(t_n, t(tokenizer.tokenize(s)))